# Encoder scaled dot-product attention

Como hemos contado el bloque de atención está compuesto por el bloque `Scaled Dot-Prooduct Attention`

<div style="text-align:center;">
  <img src="Imagenes/multi-head_attention.png" alt="Multi-Head Attention" style="width:501px;height:623px;">
</div>

De modo que vamos a ver cómo es su arquitectura

<div style="text-align:center;">
  <img src="Imagenes/Scaled_Dot-Product_Attention.png" alt="Scaled_Dot-Product_Attention">
</div>

Esta arquitectura se define también con la siguiente fórmula

<div style="text-align:center;">
  <img src="Imagenes/Scaled_Dot-Product_Attention_formula.png" alt="Scaled Dot-Product Attention formula">
</div>

Así en frío puede parecer complicado, pero vamos a ir explicando todo poco a poco para entender cómo funciona este mecanismo de atención, que constituye la parte principal de la arquitectura Transformer

## Similitud entre vectores (MatMul)

La forma que tenemos de calcular la similitud entre dos vectores consiste en calcular el producto escalar entre ambos, pero esto por qué se hace? El producto escalar entre dos vectores U y V se puede descomponer entre el producto de sus normas multiplicado por el coseno del ángulo que los separa

$$\mathbf{U} \cdot \mathbf{V} = |\mathbf{U}| \cdot |\mathbf{V}| \cos(\theta)$$

Por lo que valga lo que valga el producto de sus normas, si el ángulo que los separa es de 0º, su coseno valdrá 1, lo que quiere decir que los dos vectores son similares. Si el ángulo que los separa vale 180º, su coseno valdrá -1, lo que quiere decir que los dos vectores son opuestos o antagónicos. Pero si el ángulo que los separa es de 90º o 270º, su coseno valdrá 0, por lo que los vectores no tendrán ninguna similitud. En el siguiente gif se puede ver una muestra de este comportamiento

<div style="text-align:center;">
  <img src="Imagenes/dot-product-unity.gif" alt="dot product">
</div>

Volviendo al producto escalar, en el siguiente gif podemos ver cómo a medida que cambia el ángulo entre los vectores, el producto entre ambos cambia entre el producto de sus normas, cero o el valor negativo del producto de sus normas

<div style="text-align:center;">
  <img src="Imagenes/dot-product.gif" alt="dot product">
</div>

Ya sabemos qué hace el bloque `MatMul` de `Scaled Dot-Product Attention`, ahora vamos a explicar el bloque entero y cómo consigue calcular la atención entre distintos tokens

## Scaled dot-product attention

Vamos a volver a ver la arquitectura y la fórmula del `Scaled dot-product attention`


<div style="text-align:center;">
  <img src="Imagenes/Scaled_Dot-Product_Attention.png" alt="Scaled_Dot-Product_Attention">
  <img src="Imagenes/Scaled_Dot-Product_Attention_formula.png" alt="Scaled Dot-Product Attention formula">
</div>

Como vemos tenemos que hacer unas operaciones con `Q`, `K` y `V`. Estos nombres vienen de `query`, `key` y `value` de las bases de datos y hace una especie de similitud de lo que haces en una base de datos para obtener información, pero no vamos a hacer la explicación centrándonos en el paradigma de las bases de datos. Así que solo quédate que tienen esos nombres por esa razón

Para saber qué es `Q`, `K` y `V` volvamos a ver la arquitectura general del transformer

<div style="text-align:center;">
  <img src="Imagenes/transformer_architecture_model_encoder_multi_head_attention.png" alt="Multi-Head Attention" style="width:425px;height:626px;">
</div>

Como vemos la salida del `positional encoding`, se triplica y entra al módulo de `Multi-Head Attention`, por lo que tanto `Q`, `K` y `V` son el conjunto de vectores (es decir, la matriz) correspondientes a la frase que esté entrando al transformer

Vamos a recordar el camino de una frase cuando entra a un transformer

 1. Primero entra al tokenizador y cada palabra se divide en sub palabras más sencillas que corresponden a un ID
 2. Después entran al `Input embedding` donde son convertidas a un espacio vectorial, por lo que cada token se convertirá en un vector
 3. A continuación a cada vector se le sumará otro para dar información de la posición del token en la frase
 4. La matriz resultante se triplica y entra al `Multi-Head attention`

<div style="text-align:center;">
  <img src="Imagenes/Transformers/Slide1.jpg" alt="Multi-Head Attention">
</div>

Por tanto, tanto `Q`, como `K`, como `V` van a ser la matriz resultante del `input embedding` más el `positional encoding`. A partir de ahora, a esta matriz que se triplica la vamos a llamar `X`

En resumen, la frase de entrada al transformer, se convierte en una matriz, donde cada fila corresponde a uno de los tokens de la frase, esa matriz se triplica y entra al módulo de `Multi-Head Attention`. Por lo que en este caso, esa matriz va a ser `Q`, `K` y `V` y para simplificar la vamos a llamar `X`

### MatMul

Lo primero que hacemos es la operación `MatMul` de `Q` con `V`, pero que en realidad corresponde a la matriz `X` consigo misma

<div style="text-align:center;">
  <img src="Imagenes/Scaled_Dot-Product_Attention_first_MatMul.png" alt="MatMul">
  <img src="Imagenes/Scaled_Dot-Product_Attention_formula.png" alt="Scaled Dot-Product Attention formula">
</div>

Pero como vemos en la fórmula, para poder realizar la operación `K` tiene que estar transpuesta, veamos por qué

La matriz `X` se compone del conjunto de vectores de embeddings de la frase

$$X = \begin{pmatrix}
v_1 \\
v_2 \\
\vdots\\
v_m \\
\end{pmatrix}$$

Donde `m` es el número de tokens de la frase

Cada vector va a tener tantos elementos como las dimensiones de nuestro embedding, supongamos que es `n`, por tanto

$$X = \begin{pmatrix}
v_{1,1} & v_{1,2} & \cdots & v_{1,n} \\
v_{2,1} & v_{2,2} & \cdots & v_{2,n} \\
\vdots & \vdots & \ddots & \vdots \\
v_{m,1} & v_{m,2} & \cdots & v_{m,n} \\
\end{pmatrix}$$

Así que para poder multiplicar `X` consigo misma necesitamos que esté transpuesta

$$X \cdot X^T = \begin{pmatrix}
v_{1,1} & v_{1,2} & \cdots & v_{1,n} \\
v_{2,1} & v_{2,2} & \cdots & v_{2,n} \\
\vdots & \vdots & \ddots & \vdots \\
v_{m,1} & v_{m,2} & \cdots & v_{m,n} \\
\end{pmatrix} \cdot \begin{pmatrix}
v_{1,1} & v_{1,2} & \cdots & v_{1,m} \\
v_{2,1} & v_{2,2} & \cdots & v_{2,m} \\
\vdots & \vdots & \ddots & \vdots \\
v_{n,1} & v_{n,2} & \cdots & v_{n,m} \\
\end{pmatrix}$$

Para que así la multiplicación sea una multiplicación de matrices de dimensiones $\left(m \times n\right) \cdot \left(n \times m\right)$ que dará como resultado una matriz de tamaño $\left(m \times m\right)$ donde `m` era el número de tokens de la frase

Una vez hemos explicado por qué se tiene que multiplicar la matriz por ella misma transpuesta vamos a ver qué supone esto, si desarrollasemos la multiplicación matricial de antes llegaríamos a esto

$$X \cdot X^T = \begin{pmatrix}
v_1 \cdot v_1 & v_1 \cdot v_2 & \cdots & v_1 \cdot v_m \\
v_2 \cdot v_1 & v_2 \cdot v_2 & \cdots & v_2 \cdot v_m \\
\vdots & \vdots & \ddots & \vdots \\
v_m \cdot v_1 & v_m \cdot v_2 & \cdots & v_m \cdot v_m \\
\end{pmatrix}$$

Es decir, tendríamos una matriz con los productos escalares de los vectores de cada token de la frase, y como hemos explicado antes, el producto escalar entre dos vectores nos da la similitud entre estos. Por lo que hemos conseguido una matriz que nos da la similitud de cada token con el resto de tokens de la frase. Obviamente en la diagonal vamos a tener que la similitud va a ser el máximo, ya que se calcula la similitud de un token consigo mismo.

Vamos a ver un ejemplo con dos frases en la que se le da un hueso a un perro

 1. `I gave the dog a bone because it was hungry`
 2. `I gave the dog a bone because it was old`

En el primera frase el `it` se refiere al perro que tiene hambre, mientras que en la segunda el `it` se refiere al hueso que está viejo

In [3]:
from transformers import BertTokenizer, BertModel
import torch

In [5]:
model = BertModel.from_pretrained('bert-base-multilingual-cased')
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
sentence1 = "I gave the dog a bone because it was hungry"
sentence2 = "I gave the dog a bone because it was old"

In [10]:
tokens1 = tokenizer.tokenize(sentence1)
tokens2 = tokenizer.tokenize(sentence2)

print(f"Tokens 1: {tokens1}")
print(f"Tokens 2: {tokens2}")

Tokens 1: ['I', 'gave', 'the', 'dog', 'a', 'bone', 'because', 'it', 'was', 'hung', '##ry']
Tokens 2: ['I', 'gave', 'the', 'dog', 'a', 'bone', 'because', 'it', 'was', 'old']


In [11]:
tokens_id_1 = tokenizer.convert_tokens_to_ids(tokens1)
tokens_id_2 = tokenizer.convert_tokens_to_ids(tokens2)

print(f"Tokens ID 1: {tokens_id_1}")
print(f"Tokens ID 2: {tokens_id_2}")

Tokens ID 1: [146, 15362, 10105, 17835, 169, 57254, 12373, 10271, 10134, 68971, 10908]
Tokens ID 2: [146, 15362, 10105, 17835, 169, 57254, 12373, 10271, 10134, 12898]


In [14]:
word_embedding_1 = model(torch.tensor([tokens_id_1]))
word_embedding_2 = model(torch.tensor([tokens_id_2]))

In [15]:
type(word_embedding_1)

transformers.modeling_outputs.BaseModelOutputWithPoolingAndCrossAttentions

## Atención entre tokens con y sin el efecto del positional encoding